In [10]:
using Images
# Conway's Game of Life
# For an 8 cell neighbourhood:
#   Any live cell with fewer than two live neighbours dies, as if caused by underpopulation.
#   Any live cell with two or three live neighbours lives on to the next generation.
#   Any live cell with more than three live neighbours dies, as if by overpopulation.
#   Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.


# Cellular Automaton for Game of Life
struct CellularAutomatonGL
grid::Array
end

function CA_Print(CA::CellularAutomatonGL,savepath)
    boxsize = 5
    r,c = size(CA.grid)
    img = colorview(Gray, zeros(boxsize*r,boxsize*c))
    for row = 1:r
        row_px = 1+(row-1)*boxsize
        for col = 1:c
            col_px = 1+(col-1)*boxsize
            img[row_px:row_px+4,col_px:col_px+4] = CA.grid[row,col] == 1 ? 1 : 0
        end
    end
    save(savepath, img)
end

# Periodic Boundary conditions
function CA_periodicBC(CA::CellularAutomatonGL)
    BC = zeros(Float64,size(CA.grid).+2)
    BC[2:end-1,1] = CA.grid[:,end]
    BC[2:end-1,end] = CA.grid[:,1]
    BC[1,2:end-1] = CA.grid[end,:]
    BC[end,2:end-1] = CA.grid[1,:]
    BC[1,1] = CA.grid[end,end]
    BC[1,end] = CA.grid[end,1]
    BC[end,1] = CA.grid[1,end]
    BC[end,end] = CA.grid[1,1]
    BC[2:end-1,2:end-1] = CA.grid[:,:]
    BC
end

# Game of Life Rule
#   Initial condition is "seed"
function GL_InitialCondition(CA::CellularAutomatonGL)
    # gosper glider
    g = zeros(Float64, 9, 36)
    # left square
    g[5:6,1:2] = 1
    # right square
    g[3:4,:35:36] = 1
    # left glider
    g[5:7,11] = 1
    g[4,12] = 1
    g[8,12] = 1
    g[3,13:14] = 1
    g[9,13:14] = 1
    g[6,15] = 1
    g[4,16] = 1
    g[8,16] = 1
    g[5:7,17] = 1
    g[6,18] = 1
    # right glider
    g[3:5,21:22] = 1
    g[2,23] = 1
    g[6,23] = 1
    g[1:2,25] = 11
    g[6:7,25] = 1
    CA.grid[5:13,5:40] = g
end

#   Any live cell with fewer than two live neighbours dies, as if caused by underpopulation.
#   Any live cell with two or three live neighbours lives on to the next generation.
#   Any live cell with more than three live neighbours dies, as if by overpopulation.
#   Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.
function GL_Decision(nbhd)
    newcell = nbhd[2,2]
    nbhd[2,2] = 0
    s = sum(nbhd)
    if newcell==1
        if s<2 || s>3
            newcell = 0
        elseif s==2 || s==3
            newcell = 1
        end
    else
        newcell = s==3 ? 1 : 0
    end
    newcell
end
function GL_Rule(CA::CellularAutomatonGL)
    periodicgrid = CA_periodicBC(CA)
    nextgrid = zeros(CA.grid)
    r,c = size(nextgrid)
    nbhd = zeros(3,3)
    # Loop through current grid to update
    for row = 1:r
        for col = 1:c
            nbhd[:,:] = periodicgrid[row:row+2,col:col+2]
            newcell = GL_Decision(nbhd)
            nextgrid[row,col] = newcell
        end
    end
    CA.grid[:,:] = nextgrid[:,:]
end

function GL_Iterate(Tfinal)
    CA = CellularAutomatonGL(zeros(100,100))
    GL_InitialCondition(CA)
    CA_Print(CA,"0.jpg")
    for t = 1:Tfinal
        GL_Rule(CA)
        CA_Print(CA,"$(t).jpg")
    end
    CA
end

GL_Iterate(1000)

CellularAutomatonGL([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])